In [3]:
from __future__ import division

import gym
import tensorflow as tf
import numpy as np
import random
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt
import scipy.misc
import os
%matplotlib inline

In [5]:
class Qnetwork():
    def __init__(self,h_size):
        self.imageIn = tf.placeholder(shape=[210,160,3],dtype= tf.float32)
        self.conv1 = slim.conv2d(inputs=self.imageIn,num_outputs=32,kernal_size=[8,8],stride=[4,4],
                                 padding='SAME', biases_initializer=None)
        self.conv2 = slim.conv2d(inputs=self.conv1,num_outputs=64,kernal_size=[4,4],stride=[2,2],
                                padding='SAME', biases_initializer=None)
        self.conv3 = slim.conv2d(inputs=self.conv2,num_outputs=64,kernal_size=[3,3],stride=[1,1],
                                padding='SAME', biases_initializer=None)
        self.conv4 = slim.conv2d(inputs=self.conv3,num_outputs=h_size,kernal_size=[7,7],stride=[1,1],
                                padding='SAME', biases_initializer=None)
        
        self.streamAC,self.streamVC = tf.split(self.conv4,2,3)
        self.streamA = slim.flatten(self.steamAC)
        self.streamV = slim.flatten(self.stramVC)
        xavier_init = tf.contrib.layers.xavier_initializer()
        self.AW = tf.Variable(xavier_init([h_size//2,env.actions]))
        self.VW = tf.Variable(xavier_init([h_size//2,1]))
        self.Advantage = tf.matmul(self.streamA,self.AW)
        self.Value = tf.matmul(self.streamV,self.VW)
        
        self.Qout = self.Value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,
                                                                          keep_dims=True))
        self.predict = tf.argmax(self.Qout,1)
        
        self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions,env.actions,dtype=tf.float32)
        
        self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_onehot), axis=1)
        
        self.td_error = tf.square(self.targetQ - self.Q)
        self.loss = tf.reduce_mean(self.td_error)
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.updateModel = self.trainer.minimize(self.loss)

In [4]:
env = gym.make('MsPacman-v0')
print(env.observation_space)

Box(210, 160, 3)
